In [1]:
from neuralnilm.source import RealApplianceSource, MultiSource
import matplotlib.pyplot as plt
import numpy as np

/home/jack/workspace/python/nntools/lasagne/init.py:86: UserWarning: The uniform initializer no longer uses Glorot et al.'s approach to determine the bounds, but defaults to the range (-0.01, 0.01) instead. Please use the new GlorotUniform initializer to get the old behavior. GlorotUniform is now the default for all layers.
  warnings.warn("The uniform initializer no longer uses Glorot et al.'s "


In [2]:
real_appliance_source1 = RealApplianceSource(
    filename='/data/mine/vadeec/merged/ukdale.h5',
    appliances=[
        ['fridge freezer', 'fridge', 'freezer'],
        ['washer dryer', 'washing machine'],
        'hair straighteners',
        'television',
        'dish washer'
    ],
    max_appliance_powers=[300, 2400, 500, 200, 2500],
    on_power_thresholds=[5] * 5,
    min_on_durations=[60, 1800, 60, 60, 1800],
    min_off_durations=[12, 600, 12, 12, 1800],
    window=("2013-03-18", "2013-04-18"),
    seq_length=512,
    output_one_appliance=True,
    train_buildings=[1],
    validation_buildings=[1],
    n_seq_per_batch=8,
    skip_probability=0.75,
    skip_probability_for_first_appliance=0.5,
    target_is_start_and_end_and_mean=True
)

Loading training activations...


INFO:neuralnilm.source:Loading training activations...


  Loading activations for fridge freezer from building 1...


INFO:neuralnilm.source:  Loading activations for fridge freezer from building 1...


Loaded 760 activations.


...

INFO:neuralnilm.source:  Loading activations for dish washer from building 1...


Loaded 11 activations.


INFO:neuralnilm.source:Loaded 11 activations.


Done loading activations.


INFO:neuralnilm.source:Done loading activations.


In [3]:
real_appliance_source2 = RealApplianceSource(
    filename='/data/mine/vadeec/merged/ukdale.h5',
    appliances=[
        ['fridge freezer', 'fridge', 'freezer'],
        ['washer dryer', 'washing machine'],
        'hair straighteners',
        'television',
        'dish washer'
    ],
    max_appliance_powers=[300, 2400, 500, 200, 2500],
    on_power_thresholds=[5] * 5,
    min_on_durations=[60, 1800, 60, 60, 1800],
    min_off_durations=[12, 600, 12, 12, 1800],
    window=("2013-04-18", "2013-05-18"),
    seq_length=512,
    output_one_appliance=True,
    train_buildings=[1],
    validation_buildings=[1],
    n_seq_per_batch=8,
    skip_probability=0.75,
    skip_probability_for_first_appliance=0.5,
    target_is_start_and_end_and_mean=True
)

Loading training activations...


INFO:neuralnilm.source:Loading training activations...


  Loading activations for fridge freezer from building 1...


INFO:neuralnilm.source:  Loading activations for fridge freezer from building 1...


Loaded 763 activations.


...

INFO:neuralnilm.source:  Loading activations for dish washer from building 1...


Loaded 8 activations.


INFO:neuralnilm.source:Loaded 8 activations.


Done loading activations.


INFO:neuralnilm.source:Done loading activations.


In [4]:
multi_source = MultiSource(
    sources=[
        {
            'source': real_appliance_source1,
            'train_probability': 0.5,
            'validation_probability': 0.5
        },
        {
            'source': real_appliance_source2,
            'train_probability': 0.5,
            'validation_probability': 0.5
        }
    ],
    standardisation_source=real_appliance_source1,
    seq_length=512,
    n_seq_per_batch=8,
    n_inputs=1,
    n_outputs=1
)

In [7]:
batch = multi_source.get_batch()